In [2]:
import torch
from torch import optim, nn
from torchvision import models, transforms



ModuleNotFoundError: No module named 'torch'

In [1]:
import sys

In [3]:
print(sys.path)

['/Users/lianapatel/Git/cs348k/CS348K_FinalProject/src', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python38.zip', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/lib-dynload', '', '/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages', '/Users/lianapatel/Git/bls']


In [33]:
from ui_utils import Condition, get_dataset_subset, check_assertions

DATASET_PATH = './dataset/'
res, dataset = get_dataset_subset(DATASET_PATH + 'wimbledon_2019_womens_final_halep_williams__fduc5bZx3ss',
    ['fast', 'player_back'],5, 10, False)

assertions = [{'keypoints': ['right_elbow','right_shoulder','right_wrist', 'right_shoulder'], 'type': 'spatial', 'attributes': ['above', 'above']}, \
              {'keypoints': ['left_shoulder','right_shoulder','left_knee', 'right_knee', 'left_hip', 'right_hip'], 'type': 'spatial', 'attributes': ['left', 'left', 'left']}, \
              {'keypoints': ['left_shoulder','right_shoulder','left_knee', 'right_knee', 'left_hip', 'right_hip'], 'type': 'spatial', 'attributes': ['right', 'right', 'right']}, \
              {'keypoints': ['left_elbow','right_elbow','left_wrist','right_wrist'], 'type': 'spatial', 'attributes': [('smaller', 0.05),('smaller', 0.15)]}, \
              {'keypoints': ['left_ankle','right_ankle','left_knee','right_knee'], 'type': 'spatial', 'attributes': [('smaller', 0.05),('smaller', 0.15)]}, \
              {'keypoints': ['head_top','head_bottom'], 'type': 'spatial', 'attributes': [('smaller', 0.25)]}, \
              {'keypoints': ['right_wrist'], 'type': 'temporal', 'attributes': [0.3]},
              {'keypoints': ['left_wrist'], 'type': 'temporal', 'attributes': [0.3]}]

errors, frames = check_assertions(DATASET_PATH + 'wimbledon_2019_womens_final_halep_williams__fduc5bZx3ss', dataset, res, assertions, True)

In [36]:
image_list = [f.get_data() for f in frames]

In [38]:
# image_list = [] # TODO get from frames returned in check_assertion
image_list[0]

In [21]:
model = models.vgg16(pretrained=True)

NameError: name 'models' is not defined

In [24]:
class FeatureExtractor(nn.Module):
  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    # Extract VGG-16 Feature Layers
    self.features = list(model.features)
    self.features = nn.Sequential(*self.features)
    # Extract VGG-16 Average Pooling Layer
    self.pooling = model.avgpool
    # Convert the image into one-dimensional vector
    self.flatten = nn.Flatten()
    # Extract the first part of fully-connected layer from VGG16
    self.fc = model.classifier[0]
  
  def forward(self, x):
    # It will take the input 'x' until it returns the feature vector called 'out'
    out = self.features(x)
    out = self.pooling(out)
    out = self.flatten(out)
    out = self.fc(out) 
    return out 



In [25]:
# Initialize the model
model = models.vgg16(pretrained=True)
new_model = FeatureExtractor(model)

# Change the device to GPU if avaialble
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

NameError: name 'models' is not defined

## Feature Extraction

In [26]:
from tqdm import tqdm
import numpy as np

# Transform the image, so it becomes readable with the model
transform = transforms.Compose([
  transforms.ToPILImage(),
  transforms.CenterCrop(512),
  transforms.Resize(448),
  transforms.ToTensor()                              
])

# Will contain the feature
features = []

# Iterate each image
for data in tqdm(image_list):
    
    # Transform the image
    img = transform(data)
    
    # Reshape the image. PyTorch model reads 4-dimensional tensor
    # [batch_size, channels, width, height]
    img = img.reshape(1, 3, 448, 448)
    img = img.to(device)
    
    
    with torch.no_grad(): 
        feature = new_model(img) # Extract the feature from the image
        
    # Convert to NumPy Array, Reshape it, and save it to features variable
    features.append(feature.cpu().detach().numpy().reshape(-1))

# Convert to NumPy Array
features = np.array(features)

TabError: inconsistent use of tabs and spaces in indentation (3287425015.py, line 20)

## Clustering

In [28]:
from sklearn.cluster import KMeans

# Initialize the model
model = KMeans(n_clusters=5, random_state=42)

# Fit the data into the model
model.fit(features)

# Extract the labels
labels = model.labels_

print(labels) # [4 3 3 ... 0 0 0]

NameError: name 'features' is not defined